In [ ]:
#Package to download datasets from kaggle
#%pip install kagglehub
# %pip install fastparquet

import kagglehub
import pandas as pd
import os
import pyarrow as pa
import pyarrow.parquet as pq

# Download latest version
path = kagglehub.dataset_download("olistbr/brazilian-ecommerce")

print("Path to dataset files:", path)



In [ ]:
def data_profile(df):
    """
    Displays a quick profile of a pandas DataFrame.
    Purpose: provide a high-level overview of the dataset structure,
    data quality, and basic statistics (quick EDA).
    """

    # Shape of the DataFrame: (number of rows, number of columns)
    print(f"Data shape: {df.shape}")

    # List of column names
    print(f"\nData columns: {df.columns.tolist()}")

    # Data types of each column (int, float, object, datetime, etc.)
    print(f"\nData types: {df.dtypes}")

    # Descriptive statistics for numerical variables
    # Includes: count, mean, std, min, quartiles, and max
    print(f"\nData description: {df.describe()}")

    # Number of missing values per column
    # Helps identify data quality issues
    print(f"\nData missing values: {df.isnull().sum()}")

    # Number of duplicated rows in the DataFrame
    # Important to detect potential bias in analysis
    print(f"\nData duplicates: {df.duplicated().sum()}")


def report(df, name):
    """
    Display a quick validation report for a DataFrame.

    Purpose:
    - Confirm that the dataset has been correctly processed
    - Provide a lightweight sanity check after cleaning or transformation steps
    """

    # Display dataset name for traceability in the pipeline
    print(f"✅ {name}.csv saved!")

    # Print the DataFrame shape (rows, columns) to validate size expectations
    print("Shape:", df.shape)

    # Display the first rows to visually inspect the output
    print(df.head())


def export_clean(df, name, out_dir="data_cleaned"):
    """
    Export a cleaned DataFrame to both CSV and Parquet formats.

    Purpose:
    - CSV: human-readable format for inspection and versioning
    - Parquet: optimized columnar format for performance and type safety
    """

    # Ensure the output directory exists
    os.makedirs(out_dir, exist_ok=True)

    # Build output file paths
    csv_path = os.path.join(out_dir, f"{name}.csv")
    parquet_path = os.path.join(out_dir, f"{name}.parquet")

    # Export to CSV (universal, easy to inspect)
    df.to_csv(csv_path, index=False)

    # Export to Parquet using pyarrow
    # This approach bypasses some pandas/pyarrow compatibility issues
    # and ensures a robust Parquet write
    table = pa.Table.from_pandas(df, preserve_index=False)
    pq.write_table(table, parquet_path)

    # Confirmation message
    print(f"✅ Saved: {csv_path} and {parquet_path}")



### 1. Orders Table

#### 1.1 Load raw data




In [ ]:
#Load the orders dataset
orders_raw = pd.read_csv(os.path.join(path, "olist_orders_dataset.csv"))


#### 1.2 Profiling

- Reviewed dataset structure, columns, and data types
- Analyzed order status distribution
- Identified key timestamps for order lifecycle analysis
- Flagged early-stage statuses not relevant for delivery performance analysis

In [ ]:
# Quick data profile
data_profile(orders_raw)

#### 1.3 Data Cleaning

- Selected columns required for order lifecycle and delivery analysis
- Converted timestamp fields to datetime format
- Standardized order_status values to lowercase
- Excluded early-stage orders (`created`, `approved`)
- Applied data quality checks (primary key uniqueness, non-null status


In [ ]:
# ------------------------------------------------------------------------------
# Column selection
# Keep only the columns required for order lifecycle and delivery analysis
# Use .copy() to avoid pandas chained assignment issues (SettingWithCopyWarning)
# ------------------------------------------------------------------------------
orders_clean = orders_raw[[
    "order_id",
    "customer_id",
    "order_status",
    "order_purchase_timestamp",
    "order_delivered_customer_date",
    "order_estimated_delivery_date"
]].copy()

# ------------------------------------------------------------------------------
# Date parsing
# Convert timestamp columns to pandas datetime for time-based analysis
# Invalid or malformed values are coerced to NaT (missing)
# ------------------------------------------------------------------------------
date_cols = [
    "order_purchase_timestamp",
    "order_delivered_customer_date",
    "order_estimated_delivery_date"
]
orders_clean[date_cols] = orders_clean[date_cols].apply(pd.to_datetime, errors="coerce")

# ------------------------------------------------------------------------------
# Categorical normalization
# Standardize order_status values to lowercase to prevent case inconsistencies
# ------------------------------------------------------------------------------
orders_clean["order_status"] = orders_clean["order_status"].str.lower()

# ------------------------------------------------------------------------------
# Business rule filtering
# Exclude early-stage orders that are not relevant for delivery/performance analysis
# ------------------------------------------------------------------------------
EXCLUDED_ORDER_STATUSES = {"created", "approved"}
orders_clean = orders_clean.loc[
    ~orders_clean["order_status"].isin(EXCLUDED_ORDER_STATUSES)
]

#### 1.4 Data quality checks

In [ ]:
# ------------------------------------------------------------------------------
# Data quality checks (assertions)
# Stop the pipeline early if key assumptions are violated
# ------------------------------------------------------------------------------
assert orders_clean["order_id"].notna().all(), "order_id contains missing values"
assert orders_clean["order_id"].is_unique, "order_id is not unique"
assert orders_clean["order_status"].notna().all(), "order_status contains missing values"

#### 1.5 Export

**Output**
- `data_cleaned/orders_clean.csv`
- `data_cleaned/orders_clean.parquet`

In [ ]:
# ------------------------------------------------------------------------------
# Export cleaned dataset
# Save the cleaned table for downstream analysis and dashboarding
# ------------------------------------------------------------------------------
export_clean(orders_clean, "orders_clean")

# ------------------------------------------------------------------------------
# Sanity check report
# Quick visual validation: shape + head
# ------------------------------------------------------------------------------
report(orders_clean, "orders_clean")

### 2. Customers Table

#### 2.1 Load raw data

In [ ]:
# Load the customers dataset
customers_raw = pd.read_csv(os.path.join(path, "olist_customers_dataset.csv"))

#### 2.2 Profiling

- Inspected dataset structure and data types
- Checked for missing values
- Identified customer_id as the primary key
- Identified customer_state for regional analysis

In [ ]:
# Quick data profile
data_profile(customers_raw) 

#### 2.3 Data cleaning

- Retained only customer_id and customer_state
- Applied data quality checks (primary key uniqueness, non-null state codes)
- Validated customer_state format

In [ ]:
# ------------------------------------------------------------------------------
# Column selection
# Keep only the columns required for customer-level and regional analysis
# ------------------------------------------------------------------------------
customers_clean = customers_raw[[
    "customer_id",
    "customer_state"
]].copy()

#### 2.4 Data quality checks

In [ ]:
# ------------------------------------------------------------------------------
# Data quality checks (assertions)
# Validate key assumptions before exporting the dataset
# ------------------------------------------------------------------------------

# Primary key checks
assert customers_clean["customer_id"].notna().all(), \
    "customer_id contains missing values"
assert customers_clean["customer_id"].is_unique, \
    "customer_id is not unique"

# Categorical integrity checks
assert customers_clean["customer_state"].notna().all(), \
    "customer_state contains missing values"

# Optional: validate state code format (Brazilian states = 2-letter codes)
assert customers_clean["customer_state"].str.len().eq(2).all(), \
    "Invalid customer_state code detected"

#### 2.5 Export

**Output**
- `data_cleaned/customers_clean.csv`
- `data_cleaned/customers_clean.parquet`

In [ ]:
# ------------------------------------------------------------------------------
# Export cleaned dataset
# Ensure the output directory exists and save the cleaned table
# ------------------------------------------------------------------------------
# os.makedirs("data_cleaned", exist_ok=True)
export_clean(customers_clean, "customers_clean")

# ------------------------------------------------------------------------------
# Sanity checks
# Display basic information to validate the cleaning process
# ------------------------------------------------------------------------------
report(customers_clean, "customers_clean")

### 3. Order Items Table

#### 3.1 Load raw data





In [ ]:
order_items_raw = pd.read_csv(os.path.join(path, "olist_order_items_dataset.csv"))

#### 3.2 Profiling

- Reviewed dataset structure and data types
- Analyzed price and freight_value distributions
- Identified key columns for revenue and shipping cost analysis

In [ ]:

# ------------------------------------------------------------------------------
# Order items – Data profiling
# Explore structure, data quality, and key numerical metrics
# ------------------------------------------------------------------------------

# Generate a high-level overview of the raw order_items dataset
# (shape, columns, data types, missing values, duplicates)
data_profile(order_items_raw)

# ------------------------------------------------------------------------------
# Numerical exploration: price
# Analyze distribution and summary statistics to understand revenue patterns
# ------------------------------------------------------------------------------
print("\nPrice summary:")
print(order_items_raw["price"].describe())

# ------------------------------------------------------------------------------
# Numerical exploration: freight_value
# Analyze shipping cost distribution and identify potential outliers
# ------------------------------------------------------------------------------
print("\nFreight summary:")
print(order_items_raw["freight_value"].describe())


#### 3.3 Data Cleaning

- Selected relevant columns for analysis
- Converted price and freight_value to numeric types
- Applied data quality checks (non-null, non-negative values)


In [ ]:
# ------------------------------------------------------------------------------
# Column selection
# Keep only the columns required for revenue and shipping cost analysis
# Use .copy() to avoid chained assignment issues
# ------------------------------------------------------------------------------
order_items_clean = order_items_raw[[
    "order_id",
    "product_id",
    "price",
    "freight_value"
]].copy()

# ------------------------------------------------------------------------------
# Data type conversion
# Ensure price and freight_value are numeric for reliable aggregations
# Invalid values are coerced to NaN
# ------------------------------------------------------------------------------
order_items_clean["price"] = pd.to_numeric(
    order_items_clean["price"], errors="coerce"
)
order_items_clean["freight_value"] = pd.to_numeric(
    order_items_clean["freight_value"], errors="coerce"
)

#### 3.4 Data quality checks

In [ ]:
# ------------------------------------------------------------------------------
# Data quality checks (assertions)
# Validate business assumptions before exporting the dataset
# ------------------------------------------------------------------------------
assert order_items_clean["price"].notna().all(), "Missing values detected in price"
assert order_items_clean["freight_value"].notna().all(), "Missing values detected in freight_value"

assert (order_items_clean["price"] >= 0).all(), "Negative price values detected"
assert (order_items_clean["freight_value"] >= 0).all(), "Negative freight values detected"

#### 3.5 Export

**Output**
- `data_cleaned/order_items_clean.csv`
- `data_cleaned/order_items_clean.parquet`

In [ ]:
# ------------------------------------------------------------------------------
# Export cleaned dataset
# Save the cleaned order items table for downstream analysis
# ------------------------------------------------------------------------------
export_clean(order_items_clean, "order_items_clean")

# ------------------------------------------------------------------------------
# Sanity check report
# Quick validation of shape and sample rows
# ------------------------------------------------------------------------------
report(order_items_clean, "order_items_clean")

### 4. Products Table

#### 4.1 Load raw data




In [ ]:
# ------------------------------------------------------------------------------
# Load products dataset
# Read the raw products table from the source CSV file
# ------------------------------------------------------------------------------
products_raw = pd.read_csv(os.path.join(path, "olist_products_dataset.csv"))

#### 4.2 Profiling

- Inspected dataset structure and category distribution
- Identified product_category_name as the main analytical dimension
- Flagged descriptive and physical attributes as out of scope


In [ ]:
# ------------------------------------------------------------------------------
# Initial data profiling
# Generate a high-level overview of the dataset structure and data quality
# ------------------------------------------------------------------------------
data_profile(products_raw)

# ------------------------------------------------------------------------------
# Categorical exploration
# Analyze product categories to understand cardinality and sample values
# ------------------------------------------------------------------------------
print("\nUnique categories:", products_raw["product_category_name"].nunique())

# Display a sample of category values for quick inspection
print(products_raw["product_category_name"].unique()[:15])  # first 15 examples

#### 4.3 Data Cleaning

- Retained product_id and product_category_name
- Replaced missing categories with "unknown"
- Standardized category values
- Applied data quality checks (primary key uniqueness, non-empty categories)


In [ ]:
# ------------------------------------------------------------------------------
# Column selection
# Keep only the fields required for product-level and category analysis
# ------------------------------------------------------------------------------
products_clean = products_raw[[
    "product_id",
    "product_category_name"
]].copy()

# ------------------------------------------------------------------------------
# Missing value handling
# Replace missing product categories with "unknown" to preserve completeness
# and avoid issues during grouping or joins
# ------------------------------------------------------------------------------
products_clean["product_category_name"] = (
    products_clean["product_category_name"]
    .fillna("unknown")
    .astype(str)
    .str.strip()
    .str.lower()
)

#### 4.4 Data quality checks

In [ ]:
# ------------------------------------------------------------------------------
# Data quality checks (assertions)
# Validate key assumptions before exporting the dataset
# ------------------------------------------------------------------------------

# Primary key checks
assert products_clean["product_id"].notna().all(), "product_id contains missing values"
assert products_clean["product_id"].is_unique, "product_id is not unique"

# Category checks
assert products_clean["product_category_name"].notna().all(), \
    "product_category_name contains missing values after fillna"

# No empty strings after cleaning
assert (products_clean["product_category_name"].str.len() > 0).all(), \
    "Empty product_category_name values detected"



#### 4.5 Export

**Output**
- `data_cleaned/products_clean.csv`
- `data_cleaned/products_clean.parquet`

In [ ]:
# ------------------------------------------------------------------------------
# Export cleaned dataset
# Save the cleaned products table for downstream analysis
# ------------------------------------------------------------------------------
export_clean(products_clean, "products_clean")

# ------------------------------------------------------------------------------
# Sanity checks
# Quick validation of shape and sample rows
# ------------------------------------------------------------------------------
report(products_clean, "products_clean")

### 5. Product Category Translation Table

#### 5.1 Load raw data

In [ ]:
# ------------------------------------------------------------------------------
# Load category translation table
# Read the product category translation lookup table
# (Portuguese → English)
# ------------------------------------------------------------------------------
translation_table_raw = pd.read_csv(
    os.path.join(path, "product_category_name_translation.csv")
)

#### 5.2 Profiling

- Reviewed lookup table structure
- Confirmed absence of missing values
- Identified the table as a Portuguese-to-English category mapping

In [ ]:
# ------------------------------------------------------------------------------
# Initial data profiling
# Generate a high-level overview to validate structure and data quality
# ------------------------------------------------------------------------------
data_profile(translation_table_raw)


#### 5.3 Data Cleaning

- Renamed columns for clarity
- Standardized text fields
- Applied data quality checks (unique mapping, non-null values)


In [ ]:
# ------------------------------------------------------------------------------
# Column renaming
# Rename columns to improve clarity and semantic meaning
# ------------------------------------------------------------------------------
translation_table_clean = translation_table_raw.rename(columns={
    "product_category_name": "category_portuguese",
    "product_category_name_english": "category_english"
}).copy()

# ------------------------------------------------------------------------------
# String normalization
# Standardize text fields to reduce join mismatches (spaces/case)
# ------------------------------------------------------------------------------
translation_table_clean["category_portuguese"] = (
    translation_table_clean["category_portuguese"]
    .astype(str)
    .str.strip()
    .str.lower()
)

translation_table_clean["category_english"] = (
    translation_table_clean["category_english"]
    .astype(str)
    .str.strip()
    .str.lower()
)

#### 5.4  Data quality checks

In [ ]:
# ------------------------------------------------------------------------------
# Data quality checks (assertions)
# Validate lookup integrity before exporting the dataset
# ------------------------------------------------------------------------------

# No missing values (expected for a translation lookup table)
assert translation_table_clean["category_portuguese"].notna().all(), \
    "category_portuguese contains missing values"
assert translation_table_clean["category_english"].notna().all(), \
    "category_english contains missing values"

# No empty strings after cleaning
assert (translation_table_clean["category_portuguese"].str.len() > 0).all(), \
    "Empty category_portuguese values detected"
assert (translation_table_clean["category_english"].str.len() > 0).all(), \
    "Empty category_english values detected"

# Key uniqueness: one Portuguese category should map to one English category
assert translation_table_clean["category_portuguese"].is_unique, \
    "Duplicate category_portuguese detected (mapping should be 1-to-1)"

# Optional: sanity check for duplicates on the full pair (PT, EN)
assert translation_table_clean.duplicated(
    subset=["category_portuguese", "category_english"]
).sum() == 0, "Duplicate translation pairs detected"

#### 5.5 Export

**Output**
- `data_cleaned/category_translation_clean.csv`
- `data_cleaned/category_translation_clean.parquet`

In [ ]:
# ------------------------------------------------------------------------------
# Export cleaned translation table
# Ensure the output directory exists and save the cleaned lookup table
# ------------------------------------------------------------------------------
export_clean(translation_table_clean, "translation_table_clean")

# ------------------------------------------------------------------------------
# Sanity checks
# Quick validation of shape and sample rows
# ------------------------------------------------------------------------------
report(translation_table_clean, "translation_table_clean")

### 6. Reviews Table

#### 6.1 Load raw data

In [ ]:
reviews_table_raw = pd.read_csv(
    os.path.join(path, "olist_order_reviews_dataset.csv")
)


#### 6.2 Profiling

- Reviewed dataset structure, columns, and data types
- Identified `order_id` as the join key with the orders table
- Analyzed the distribution of `review_score` to assess customer satisfaction patterns
- Verified that review scores follow a discrete rating scale

In [ ]:
data_profile(reviews_table_raw)

print("\nReview score distribution:")
print(reviews_table_raw['review_score'].value_counts())

#### 6.3 Data Cleaning

- Selected only the columns required for satisfaction analysis:
  - `order_id`
  - `review_score`
- Ensured `review_score` values are numeric


In [ ]:
reviews_table_clean = reviews_table_raw[["order_id", "review_score"]].copy()


#### 6.4 Data quality checks

- Applied data quality checks:
  - non-null `order_id`
  - non-null `review_score`
  - review scores constrained to the expected range (1–5)

In [ ]:
# ------------------------------------------------------------------------------
# Data quality checks (assertions)
# Validate assumptions for reviews data
# ------------------------------------------------------------------------------

# order_id must be present for joins with orders
assert reviews_table_clean["order_id"].notna().all(), \
    "order_id contains missing values in reviews table"

# review_score must be present
assert reviews_table_clean["review_score"].notna().all(), \
    "review_score contains missing values"

# review_score must be numeric
assert pd.api.types.is_numeric_dtype(reviews_table_clean["review_score"]), \
    "review_score is not numeric"

# review_score must be within expected rating scale (1 to 5)
assert reviews_table_clean["review_score"].between(1, 5).all(), \
    "review_score outside expected range (1–5)"


#### 5.5 Export

**Output**
- `data_cleaned/reviews_table_clean.csv`
- `data_cleaned/reviews_table_clean.parquet`

In [ ]:
# ------------------------------------------------------------------------------
# Export cleaned translation table
# Ensure the output directory exists and save the cleaned lookup table
# ------------------------------------------------------------------------------
export_clean(reviews_table_clean, "reviews_table_clean")

# ------------------------------------------------------------------------------
# Sanity checks
# Quick validation of shape and sample rows
# ------------------------------------------------------------------------------
report(reviews_table_clean, "reviews_table_clean")